In [34]:
import pandas as pd
import lightgbm as lgb
import os
import gc
import pickle
from sklearn import preprocessing, model_selection

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder,StandardScaler
from tensorflow.keras.utils import to_categorical

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [35]:
#"""
def get_type_feature_all_(sample, train_df, key, on, type_c, mark):
    if type_c == "count":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].count()).reset_index()
    if type_c == "mean":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].mean()).reset_index()
    if type_c == "nunique":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].nunique()).reset_index()
    if type_c == "max":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].max()).reset_index()
    if type_c == "min":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].min()).reset_index()
    if type_c == "sum":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].sum()).reset_index()
    if type_c == "std":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].std()).reset_index()
    if type_c == "median":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].median()).reset_index()
    tmp.columns = key + [mark + "_" + "_".join(key) + '_%s_' % type_c + on]
    tmp[mark + "_" + "_".join(key) + '_%s_' % type_c + on] = tmp[
        mark + "_" + "_".join(key) + '_%s_' % type_c + on].astype('float32')
    sample = sample.merge(tmp, on=key, how='left')
    del tmp, train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on
#"""
def get_type_feature_all(sample,train_df,key,on,type_c,mark):
    pickle_path="../pickle/"
    filename = "_".join([mark+"_%s_features"%type_c, "_".join(key), on, str(len(sample))]) + ".pkl"
    try:
        with open(pickle_path + filename, "rb") as fp:
            print("load {} {} feature from pickle file: key: {}, on: {}...".format(mark,type_c,"_".join(key), on))
            col = pickle.load(fp)
        for c in col.columns:
            sample[c] = col[c]
        gc.collect()
    except:
        print('get {} {} feature, key: {}, on: {}'.format(mark,type_c,"_".join(key), on))
        if type_c=="count":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].count()).reset_index()
        if type_c=="mean":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].mean()).reset_index()
        if type_c=="nunique":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].nunique()).reset_index()
        if type_c=="max":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].max()).reset_index()
        if type_c=="min":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].min()).reset_index()
        if type_c=="sum":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].sum()).reset_index()
        if type_c=="std":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].std()).reset_index()
        if type_c=="median":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].median()).reset_index()
        tmp.columns = key+[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
        tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on] = tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on].astype('float32')
        sample = sample.merge(tmp, on=key, how='left')
        with open(pickle_path + filename, "wb") as fp:
            col = sample[[mark+"_"+"_".join(key) + '_%s_'%type_c + on]].copy()
            pickle.dump(col, fp)
        del tmp
    #if save_only:
        #for i in col.columns:
            #del sample[i]
    del col,train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on


In [36]:
path="../data/"
train_data=pd.read_csv(path+"train.csv")
test_data=pd.read_csv(path+"test.csv")

In [37]:
train_data.columns

Index(['article_id', 'date', 'baike_id_1h', 'price', 'price_diff', 'author',
       'level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url',
       'comments_1h', 'zhi_1h', 'buzhi_1h', 'favorite_1h', 'orders_1h',
       'baike_id_2h', 'comments_2h', 'zhi_2h', 'buzhi_2h', 'favorite_2h',
       'orders_2h', 'orders_3h_15h'],
      dtype='object')

In [15]:
def create_features(data):
    data["order_1h_2h"]=data["orders_1h"]+data["orders_2h"]
    
    date_max=data.groupby(["date"]).article_id.max().reset_index()
    date_max.columns=["date","date_max"]
    data_min=date_max.copy()
    data_min.columns=["date","date_min"]
    data_min["date"]=data_min["date"]+1
    
    data=data.merge(date_max,on="date",how="left")
    data=data.merge(data_min,on="date",how="left")
    data["date_min"]=data["date_min"].fillna(0)
    
    data["article_id"]=(data["article_id"]-data["date_min"])/(data["date_max"]-data["date_min"])
    
    data['hour']=data["article_id"].apply(lambda x:int(x*24))
    
    data["dayofweek"]=data["date"]%7
    
    del data["date_min"],data["date_max"]
    
    
    cat_features=[['baike_id_1h'],['author'],['level1'], ['level2'], ['level3'], ['level4'], ['brand'], ['mall'], ['url'],['dayofweek'],
                  ['author','level1'],['author','level2'],['author','level3'],['author','level4'],['author','brand'],['author','mall'],
                  ['mall','brand'],['mall','level1'],['mall','level2'],['mall','level3'],['mall','level4'],
                  ['brand','level1'],['brand','level2'],['brand','level3'],['brand','level4'],
                  ['baike_id_1h','mall'],
                  ["mall","brand","level1"],["mall","brand","level2"],["mall","brand","level3"],["mall","brand","level4"]

                 ]
    num_features=['comments_1h','comments_2h',
                  'zhi_1h','zhi_2h',
                  'buzhi_1h','buzhi_2h',
                  'favorite_1h','favorite_2h',
                  'orders_1h','orders_2h','order_1h_2h','article_id']
    
    for cat in cat_features:
        data["%s_%s_-1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-1)-data["article_id"]
        data["%s_%s_1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(1)-data["article_id"]

        data,_=get_type_feature_all(data, data, cat,"%s_%s_-1"%("_".join(cat),"article_id") , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"%s_%s_1"%("_".join(cat),"article_id") , "mean", "fe")
    
    for cat in cat_features:
        data,_=get_type_feature_all(data, data, cat+["hour"],"article_id" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"article_id" , "count", "fe")
        
        data,_=get_type_feature_all(data, data, cat,"price" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "max", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "min", "fe")
        

        for num in num_features:
            data,_=get_type_feature_all(data, data, cat,num , "mean", "fe")
            data,_=get_type_feature_all(data, data, cat,num , "sum", "fe")
    
    return data

In [16]:
data=pd.concat([train_data,
                #test_data,
               ]).reset_index(drop=True)

data=create_features(data)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pickle file: key: brand_hour, on: article_id...
load fe count feature f

load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: author_level1, on: comments_1h...
load fe sum feature from pickle file: key:

load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe mean feature from pickle file: key: author_level4, on: article_id...
load fe sum feature from pickle file: key: author_level4, on: article_id...
load f

load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_level1, on: orders_1h...
load fe mean feature from pickle file: key: mall_level1, on: orders_2h...
load fe sum feature from pickle file: key: mall_level1, on: orders_2h...
load fe mean feature from pickle file: key: 

load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_1h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_2h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_2h...
load fe mean feature from pickle file:

load fe mean feature from pickle file: key: baike_id_1h_mall, on: price...
load fe max feature from pickle file: key: baike_id_1h_mall, on: price...
load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h..

load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle file: key: mall_brand_level3, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4_hour, on: article_id...
load fe count feature from pickle file: key: mall_brand_level4, on: article_id...
load fe mean feature from p

In [17]:
print(data.shape)

(1807657, 1018)


In [18]:
train=data[(data.date<110)].copy()
#train=data[data.date<117].copy()
#train=data[(data.date>=103)&(data.date<110)].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()

In [19]:
features = list(train[:1].drop(['date', 'orders_3h_15h'],axis=1).columns)

In [20]:
train_ind=data[data.date<110].index
valid_ind=data[(data.date>=110)&(data.date<117)].index
test_ind=data[data.date>=117].index

data_x=data[features].fillna(0).values

nor=StandardScaler()
data_x = nor.fit_transform(data_x)

train_x = data_x[train_ind]
valid_x = data_x[valid_ind]
test_x = data_x[test_ind]

train_y = data["orders_3h_15h"].values[train_ind].astype('int')
valid_y = data["orders_3h_15h"].values[valid_ind].astype('int')
train_y=to_categorical(train_y)
valid_y=to_categorical(valid_y)

In [21]:
def get_model():
    input_num = Input(shape=(len(features),))
    
    fc = Dense(2048,activation='relu')(input_num)
    fc = BatchNormalization()(fc)

    fc = Dense(1024,activation='relu')(fc)
    fc = BatchNormalization()(fc)

    fc = Dense(512,activation='relu')(fc)
    fc = BatchNormalization()(fc)
    
    fc = Dense(256,activation='relu')(fc)
    fc = BatchNormalization()(fc)
    
    preds = Dense(10,activation='softmax')(fc)

    model = Model(inputs=input_num, outputs=preds)
    return model

In [1]:
model=get_model()
opt = Adam(lr=0.001, clipnorm=2., amsgrad=False)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=[])

early_stop = EarlyStopping(patience=2)
check_point = ModelCheckpoint('best_model.hdf5', monitor="val_loss", mode="min", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=1, min_lr=0.0001)
model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                    batch_size=1024, epochs=10, verbose=1,#shuffle=False,
                    callbacks=[early_stop, check_point,reduce_lr]
                   )
model.load_weights('best_model.hdf5')

NameError: name 'get_model' is not defined

In [23]:
model.predict(valid_x,batch_size=1024).shape

(144961, 10)

In [24]:
sub_id=train_data[(train_data.date>=110)&(train_data.date<117)][["article_id"]].copy()
sub=pd.DataFrame(model.predict(valid_x,batch_size=1024))
sub.columns=["nn_multiclass_%s"%i for i in range(10)]
sub["article_id"]=list(sub_id["article_id"])
sub.to_csv("../user_data/train_stacking_nn_multiclass.csv",index=None)

In [25]:
sub

,nn_multiclass_0,nn_multiclass_1,nn_multiclass_2,nn_multiclass_3,nn_multiclass_4,nn_multiclass_5,nn_multiclass_6,nn_multiclass_7,nn_multiclass_8,nn_multiclass_9,article_id
0,0.038238,0.072579,0.065181,0.063791,0.300779,0.316147,0.122056,0.020555,5.837420e-04,8.900379e-05,1664040
1,0.841858,0.104751,0.029198,0.011309,0.011765,0.000987,0.000110,0.000017,3.105043e-06,2.484665e-06,1664289
2,0.024386,0.045198,0.039160,0.042686,0.240643,0.386434,0.182649,0.037515,1.167294e-03,1.619879e-04,1664686
3,0.782518,0.136013,0.041333,0.016968,0.020363,0.002434,0.000310,0.000049,6.376859e-06,4.916004e-06,1666137
4,0.657972,0.184915,0.075970,0.033779,0.044441,0.002728,0.000170,0.000021,2.370991e-06,1.956832e-06,1667990
...,...,...,...,...,...,...,...,...,...,...,...
144956,0.885163,0.083870,0.019858,0.006046,0.004831,0.000198,0.000023,0.000005,2.924535e-06,2.719958e-06,1804751
144957,0.709741,0.176547,0.051120,0.023179,0.034372,0.004453,0.000512,0.000069,4.535916e-06,3.297379e-06,1805971
144958,0.922817,0.064702,0.008519,0.002427,0.001399,0.000110,0.000021,0.000004,6.595984e-07,4.157306e-07,1807148
144959,0.930537,0.055937,0.008731,0.002678,0.002013,0.000084,0.000014,0.000004,1.537918e-06,1.168459e-06,1807424


In [26]:
data=pd.concat([train_data,
                test_data,
               ]).reset_index(drop=True)
data=create_features(data)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe mean feature from pickle file: key: level4, on: buzhi_2h...
load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pic

load fe mean feature from pickle file: key: dayofweek, on: orders_1h...
load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: auth

load fe sum feature from pickle file: key: author_level4, on: buzhi_1h...
load fe mean feature from pickle file: key: author_level4, on: buzhi_2h...
load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe me

load fe mean feature from pickle file: key: mall_level1, on: zhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: zhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: zhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_leve

load fe mean feature from pickle file: key: brand_level1, on: comments_1h...
load fe sum feature from pickle file: key: brand_level1, on: comments_1h...
load fe mean feature from pickle file: key: brand_level1, on: comments_2h...
load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file:

load fe sum feature from pickle file: key: brand_level4, on: article_id...
load fe mean feature from pickle file: key: baike_id_1h_mall_hour, on: article_id...
load fe count feature from pickle file: key: baike_id_1h_mall, on: article_id...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: price...
load fe max feature from pickle file: key: baike_id_1h_mall, on: price...
load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, o

load fe sum feature from pickle file: key: mall_brand_level3, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle fi

In [27]:
train=data[data.date<117].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()
test=data[data.date>=117].copy()

In [28]:
train_ind=data[data.date<117].index
valid_ind=data[(data.date>=110)&(data.date<117)].index
test_ind=data[data.date>=117].index

data_x=data[features].fillna(0).values

nor=StandardScaler()
data_x = nor.fit_transform(data_x)

train_x = data_x[train_ind]
valid_x = data_x[valid_ind]
test_x = data_x[test_ind]

train_y = data["orders_3h_15h"].values[train_ind].astype('int')
valid_y = data["orders_3h_15h"].values[valid_ind].astype('int')
train_y=to_categorical(train_y)
valid_y=to_categorical(valid_y)

In [31]:
sub_id=test_data[["article_id"]].copy()
sub=pd.DataFrame(model.predict(test_x,batch_size=1024))
sub.columns=["nn_multiclass_%s"%i for i in range(10)]
sub["article_id"]=list(sub_id["article_id"])
sub.to_csv("../user_data/test_stacking_nn_multiclass.csv",index=None)

In [33]:
sub

,nn_multiclass_0,nn_multiclass_1,nn_multiclass_2,nn_multiclass_3,nn_multiclass_4,nn_multiclass_5,nn_multiclass_6,nn_multiclass_7,nn_multiclass_8,nn_multiclass_9,article_id
0,0.876106,0.081713,0.022513,0.009198,0.009352,0.000978,1.168082e-04,1.671348e-05,3.830202e-06,3.003257e-06,1809360
1,0.511773,0.239785,0.099180,0.049569,0.081381,0.015805,2.200869e-03,2.831376e-04,1.569330e-05,7.165844e-06,1809857
2,0.668786,0.224630,0.060060,0.020728,0.022743,0.002593,3.731007e-04,7.535380e-05,7.250600e-06,5.475916e-06,1810275
3,0.039412,0.062027,0.057364,0.057865,0.270282,0.347693,1.398233e-01,2.467558e-02,7.280813e-04,1.305605e-04,1810622
4,0.913253,0.062927,0.014144,0.005088,0.004079,0.000417,7.172829e-05,1.756130e-05,1.973997e-06,1.024473e-06,1811139
...,...,...,...,...,...,...,...,...,...,...,...
149584,0.969638,0.026855,0.002576,0.000586,0.000319,0.000018,4.240306e-06,1.849827e-06,9.519234e-07,9.374843e-07,1955932
149585,0.993462,0.003309,0.001448,0.000570,0.001072,0.000095,2.618251e-05,1.236512e-05,3.377549e-06,2.527071e-06,1956580
149586,0.997214,0.002181,0.000456,0.000088,0.000056,0.000002,7.823654e-07,4.852192e-07,3.563976e-07,1.807855e-07,1956675
149587,0.995735,0.003783,0.000364,0.000066,0.000046,0.000002,8.503348e-07,5.446245e-07,9.223451e-07,1.229292e-06,1956922
